# Protein-Protein Interaction
Here we work with Ece's list of gene to define protein-protein interactions

In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import requests
import json
from pp_interactions import fetch_string_interactions, fetch_biogrid_interactions, visualize_network, get_gene_symbol_uniprot, get_uniprot_id

In [2]:
# Load dataset
df = pd.read_excel("Input/ece_secpath_candidates.xlsx")
candidates = list(df['Genes'])

## STRINGS

In [ ]:
if __name__ == "__main__":
    for gene in candidates:
        # Replace with your list of genes
        gene_list = [gene,'GM-CSF', 'IL21', 'IL15']

        # Fetch and visualize STRING interactions
        G_string = fetch_string_interactions(gene_list)
        print(f"Visualizing STRING interactions for {gene_list}...")
        
        filename = f"Plots/STRING_interactions_{gene}.png"
        visualize_network(G=G_string, gene_list=gene_list, node_size=200, labels_size=11, filename=filename) 

In [ ]:
if __name__ == "__main__":
    
    cytokines = ['GM-CSF', 'IL21', 'IL15']
    gene_list = candidates + cytokines 

    # Fetch and visualize STRING interactions
    G_string = fetch_string_interactions(gene_list)
    print("Visualizing STRING interactions...")
    visualize_network(G=G_string, gene_list=gene_list, node_size=150, labels_size=9)  # Pass gene_mapping here

## Biogrid

In [ ]:
url = "https://webservice.thebiogrid.org/interactions/?search=IL21&accesskey=0659585a790acc50563e75a55f72b15f&format=json"
response = requests.get(url)
data = response.json()

In [ ]:
data

In [ ]:
data['103']

In [ ]:
gene_list = ['BRCA1']

In [ ]:
G = fetch_biogrid_interactions(gene_list)

In [ ]:
for g in G.nodes():
    print(g)

In [ ]:
visualize_network(G=G, gene_list=gene_list, node_size=150, labels_size=9)  # Pass gene_mapping here

## Genemania
Protein-protein interaction and co-expression

## PrePPI
https://www.sciencedirect.com/science/article/pii/S0022283623001080?via%3Dihub
https://honiglab.c2b2.columbia.edu/PrePPI/

In [4]:
candidates_uniprot = get_uniprot_id(candidates)

In [5]:
candidates_uniprot

{'MGAT5': 'Q8R4G5',
 'ARHGEF2': 'O43353',
 'HSPA8': 'Q9UNE7',
 'RAB20': 'Q9GSZ2',
 'TBC1D20': 'Q9H0U4',
 'STX11': 'Q15833',
 'GALNT10': 'Q4G0E1',
 'ARFGAP1': 'Q63524',
 'GALNT3': 'A0A812PEB8',
 'JUN': 'K7RMT3',
 'PGAP1': 'A0A0S7LHE3',
 'VAMP5': 'F7DCD3',
 'CYTH4': 'L8E9K5',
 'DNAJC6': 'L0N9S0',
 'AGAP1': 'Q8BSZ2',
 'PSD3': 'A0A0S7IS93',
 'RAB3B': 'Q9UNE2',
 'DPM1': 'Q9P2X0',
 'TMED7': 'A0A0S7JF52',
 'RPN2': 'Q9D328',
 'PIGW': 'A0A0S7EIU5',
 'MON1A': 'P86791',
 'TMED2': 'Q63584',
 'TBC1D24': 'P62330',
 'SRP14': 'P49458',
 'SRP9': 'P16254',
 'SRP54': 'Q5R4R6',
 'SRPRa': 'A0A024B7W1',
 'SRPRb': 'P08240',
 'SEC61a1': 'P60059',
 'SEC61B': 'O43681',
 'SEC61G': 'Q5JPE7',
 'DNMBP': 'Q03173',
 'AGAP2': 'O88801',
 'AP1S2': 'P15941',
 'B4GALT1': 'P29752',
 'TBC1D1': 'Q8K3G9',
 'CYTH1': 'O60759',
 'ARHGEF11': 'Q8N6F7',
 'ARAP2': 'A0A0S7LFT1',
 'EIF2AK2': 'O75569',
 'USP6NL': 'Q96FW8',
 'NAPA': 'P0A9I5',
 'RAB12': 'Q96CV9',
 'OPTN': 'Q9UHD2',
 'HSPA1B': 'Q8WXX5',
 'RAB3IP': 'P63012',
 'TLR2': 'Q153

In [ ]:
file_path = 'Input/preppi_interactome/preppi.human_af.interactome.txt'
df = pd.read_csv(file_path, sep='\t')

In [ ]:
df

In [ ]:
prots = list(df['prot2'][0:100])

In [ ]:
for prot in prots:
    gene = get_gene_symbol_uniprot(prot)
    print(prot, gene)

In [ ]:
genes